In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import numpy as np
import pandas as pd
import scipy.optimize

from datetime import datetime
import pytz

import gm2
import trfp

In [ ]:
def trolley_run_station_average(corrected_df):
    station_phi = trfp.STATION_BARCODE_PHI
    station_edges = trfp.STATION_BARCODE_EDGES

    # tr_phi is not monotonic, so sort by tr_phi

    corrected_df = corrected_df.sort_values(by=['tr_phi'])

    measured_phi = corrected_df['tr_phi'].values
    measured_extent = (np.roll(measured_phi,-1)-np.roll(measured_phi,1))/2
    measured_extent[0] = measured_extent[0]+180
    measured_extent[-1] = measured_extent[-1]+180
    # print np.max(measured_extent)

    corrected_df['tr_extent'] = pd.Series(measured_extent, index=corrected_df.index)
    corrected_df = corrected_df.sort_index()

    # for a given station:
    # create a mask for when trolley is in [low edge, high edge)
    tr_baseline = np.full((72,17), np.nan)
    fp_baseline = np.full((72,6), np.nan)
    summed_azimuth = np.full(72, np.nan)
    summed_pts = np.full(72, np.nan)
    baseline_time = np.full(72, np.nan)

    for st in range(72): 
        if station_edges[st+1] > station_edges[st]:
            mask = (corrected_df['tr_phi'] >= station_edges[st]) & (corrected_df['tr_phi'] < station_edges[st+1])
        else:  # case where we go over the 360 deg line
            mask = (corrected_df['tr_phi'] >= station_edges[st]) | (corrected_df['tr_phi'] < station_edges[st+1])

        out_df = corrected_df[mask]
        summed_pts[st] = out_df.shape[0]
        summed_azimuth[st] = sum(out_df['tr_extent'].values)        
        baseline_time[st] = sum(out_df.index.values)/summed_pts[st]

        for m in range(17):
            st_id = 'tr,m'+str(m+1)
            if sum(out_df['tr_extent'].values) != 0:
                tr_baseline[st, m] = sum(out_df['tr_extent'].values*out_df[st_id].values)/sum(out_df['tr_extent'].values)
            else:
                tr_baseline[st, m] = np.nan
        for m in range(6):
            st_id = 'st'+str(st)+',m'+str(m+1)
            if sum(out_df['tr_extent'].values) != 0:
                fp_baseline[st, m] = np.mean(out_df[st_id])
            else:
                fp_baseline[st, m] = np.nan
    
    return tr_baseline, fp_baseline, baseline_time, summed_azimuth, summed_pts

def remove_trolley_effect(trolley_moment_df):
    '''DOC STRING'''
    barcode = trfp.STATION_BARCODE_PHI
    veto_extent = 25

    trolley_effect_removed_df = trolley_moment_df.copy()

    for st in range(72):
        print '\rRemoving trolley image from station ' + str(st) + '.',
        for m in range(1,7):
            st_m = 'st' + str(st) + ",m" + str(m)

            # Unwrap the fixed probe data versus trolley position
            raw_data = trolley_moment_df[['tr_phi', st_m]].copy()
            raw_low = raw_data.copy()
            raw_high = raw_data.copy()
            raw_low['tr_phi'] = raw_low['tr_phi'] - 360
            raw_high['tr_phi'] = raw_high['tr_phi'] + 360
            unwrap_nomask_df = pd.concat([raw_low, raw_data, raw_high])

            unwrap_mask_df = unwrap_nomask_df.copy()
            veto_adjust = (veto_extent-7)/2
            mask = ((unwrap_nomask_df['tr_phi']>barcode[st]-2-veto_adjust) & (unwrap_nomask_df['tr_phi']<barcode[st]+5+veto_adjust) |
                    (unwrap_nomask_df['tr_phi']>barcode[st]-2-veto_adjust-360) & (unwrap_nomask_df['tr_phi']<barcode[st]+5+veto_adjust-360) |
                    (unwrap_nomask_df['tr_phi']>barcode[st]-2-veto_adjust+360) & (unwrap_nomask_df['tr_phi']<barcode[st]+5+veto_adjust+360))
            unwrap_mask_df[st_m] = unwrap_nomask_df[st_m].mask(mask)
            unwrap_mask_df['tr_phi'] = unwrap_nomask_df['tr_phi']

            unwrap_filled_df = unwrap_mask_df.copy()
            temp = unwrap_filled_df.rolling(int(500),win_type='triang',min_periods=1,center=True).mean()
            temp = temp.rolling(int(500),win_type='triang',min_periods=1,center=True).mean()
            unwrap_filled_df[st_m] = unwrap_filled_df[st_m].mask(mask, temp[st_m])

            length = raw_data.shape[0]
            filled_df = unwrap_filled_df.iloc[length:2*length,:]

            trolley_effect_removed_df[st_m] = filled_df[st_m]

    print '\rFinished removing trolley images from ' + str(length) + ' events.'
    return trolley_effect_removed_df


In [ ]:
station_phi = trfp.STATION_BARCODE_PHI
station_edges = trfp.STATION_BARCODE_EDGES

corrected_df_1 = remove_trolley_effect(pd.read_hdf('60hr_trolley_runs_1.h5', key='run_3956_moment_df'))
tr_baseline_1, fp_baseline_1, baseline_time_1, summed_azimuth_1, _ = trolley_run_station_average(corrected_df_1)

corrected_df_2 = remove_trolley_effect(pd.read_hdf('60hr_trolley_runs_2.h5', key='run_3997_moment_df'))
tr_baseline_2, fp_baseline_2, baseline_time_2, summed_azimuth_2, _ = trolley_run_station_average(corrected_df_2)

# load all fixed probe runs moment_df into one big moment_df (runs 3959--3994)
print 'Appending fixed probe runs.'
fp_moment_df = pd.read_hdf('60hr_fixed_probe_runs_new.h5', key='run_3959_moment_df')
pts = fp_moment_df.shape[0]
for run in np.arange(3959, 3995):
    temp_df = pd.read_hdf('60hr_fixed_probe_runs_new.h5', key='run_'+str(run)+'_moment_df')
    pts = pts + temp_df.shape[0]
    print '\rAppending run ' + str(run) + '.',
    fp_moment_df = fp_moment_df.append(temp_df)

# load all the trolley runs corrected_df into on big file (to show window of 60hr set) ()
print '\nAppending trolley runs.'
tr_corrected_df = remove_trolley_effect(pd.read_hdf('60hr_trolley_runs_1.h5', key='run_3955_moment_df'))
for run in np.arange(3956, 3959):
    temp_df = remove_trolley_effect(pd.read_hdf('60hr_trolley_runs_1.h5', key='run_'+str(run)+'_moment_df'))
    pts = pts + temp_df.shape[0]
    print '\rAppending run ' + str(run) + '.',
    tr_corrected_df = tr_corrected_df.append(temp_df)
for run in np.arange(3995, 3999):
    temp_df = remove_trolley_effect(pd.read_hdf('60hr_trolley_runs_2.h5', key='run_'+str(run)+'_moment_df'))
    pts = pts + temp_df.shape[0]
    print '\rAppending run ' + str(run) + '.',
    tr_corrected_df = tr_corrected_df.append(temp_df)

print '\nDone appending runs.'


In [ ]:
### Apply baseline corrections, generate virtual trolley measurements

fp_moment_baseline = fp_moment_df.copy()
fp_moment_tr_run_baseline = tr_corrected_df.copy()

# apply baseline corrections to each fp stm
print "\nSubtracting fixed probe baselines."
for st in np.arange(72):
    num_probes = len(trfp.STATION_PROBE_ID[st])
    for m in np.arange(num_probes):
        stm = 'st'+str(st)+',m'+str(m+1)
        
        def backwards_correction(time):
            c1 = fp_baseline_1[st, m]
            c2 = fp_baseline_2[st, m]
            t1 = baseline_time_1[st]
            t2 = baseline_time_2[st]
            return (c2-c1)/(t2-t1)*(time-t1) + c1
        
        correction = backwards_correction(fp_moment_baseline.index.values)
        fp_moment_baseline[stm] = fp_moment_baseline[stm] - correction
        correction = backwards_correction(fp_moment_tr_run_baseline.index.values)
        fp_moment_tr_run_baseline[stm] = fp_moment_tr_run_baseline[stm] - correction
        
        print '\rstm: ' + stm + '.',
        
# replace columns in vtr with Jacobian-fixed columns from fp baseline correction

print "\n\nApplying Jacobian."
vtr_df = fp_moment_baseline.copy()
vtr_tr_run_df = fp_moment_tr_run_baseline.copy()

for st in np.arange(72):
    num_probes = len(trfp.STATION_PROBE_ID[st])
    if num_probes == 4:
        num_moments = 4
        if st == 41:
            J = trfp.J_4_PROBE_ST41
        elif st == 37 | st == 39:
            J = trfp.J_4_PROBE_ST37_ST39
        else:
            J = trfp.J_4_PROBE
    else:
        num_moments = 5
        if st < 7:
            J = trfp.J_6_PROBE_OFFSET
        else:
            J = trfp.J_6_PROBE
#         J[0,4] = 0

    # run over each vtr moment:
    for m in np.arange(num_moments):
        vtr_stm = 'st'+str(st)+',m'+str(m+1)
        fp_stm = ['st'+str(st)+',m'+str(fp_m+1) for fp_m in np.arange(num_moments)]
        vtr_df[vtr_stm] = fp_moment_baseline[fp_stm].dot(J[m])
        vtr_tr_run_df[vtr_stm] = fp_moment_tr_run_baseline[fp_stm].dot(J[m])
        
        print '\rstm: ' + stm + '.',
        
# Add trolley baseline correction (with backwards correction)
print "\n\nAdding trolley baselines."

for st in np.arange(72):
    num_probes = len(trfp.STATION_PROBE_ID[st])
    for m in np.arange(num_probes):
        stm = 'st'+str(st)+',m'+str(m+1)
        
        def backwards_correction(time):
            c1 = tr_baseline_1[st, m]
            c2 = tr_baseline_2[st, m]
            t1 = baseline_time_1[st]
            t2 = baseline_time_2[st]
            return (c2-c1)/(t2-t1)*(time-t1) + c1
        
        correction = backwards_correction(vtr_df.index.values)
        vtr_df[stm] = vtr_df[stm] + correction
        correction = backwards_correction(vtr_tr_run_df.index.values)
        vtr_tr_run_df[stm] = vtr_tr_run_df[stm] + correction
        
        print '\rstm: ' + stm + '.',

print '\n\nDone.'


In [ ]:
# bin into the agreed upon bins

bins = np.arange(1524384055, 1524641055, 1000)-500
bin_centers = np.arange(1524384055, 1524640055, 1000)

vtr_time_bin_df = vtr_df.groupby(pd.cut(vtr_df.index,bins)).mean()
vtr_time_bin_df.index = bin_centers

fp_time_bin_df = fp_moment_df.groupby(pd.cut(fp_moment_df.index,bins)).mean()
fp_time_bin_df.index = bin_centers

vtr_time_bin_df.head()
test_df = vtr_time_bin_df.copy()

azi_avg_df = pd.DataFrame(np.zeros((test_df.shape[0],6)),
                         index = test_df.index,
                         columns = ['m' + str(m) for m in np.arange(6)+1])

weight = summed_azimuth_1+summed_azimuth_2
total_weight = np.sum(weight)

for m in np.arange(6):
    stm_list = ['st'+str(st)+',m'+str(m+1) for st in np.arange(72)]
    azi_avg_df['m'+str(m+1)] = test_df[stm_list].multiply(weight).sum(axis=1)/total_weight
    
azi_avg_df.head()

fig, axs = plt.subplots(2,1)

plt.sca(axs[0])
plt.plot(azi_avg_df.index.values, azi_avg_df['m2']/61.79, '.')

fig.set_size_inches(12,8)
fig.tight_layout()

# azi_avg_df.to_csv('no_sext_correction')

# This section was trying to find the issue with the n quad

Turns out that, somehow, the hdf5 files got overwritten, and most of the 4-probe m2 moments were bad.

In [ ]:
print np.round(trfp.J_6_PROBE[1,:],2)
print np.round(trfp.J_6_PROBE_OFFSET[1,:],2)
print np.round(trfp.J_4_PROBE[1,:],2)
print np.round(trfp.J_4_PROBE_ST37_ST39[1,:],2)
print np.round(trfp.J_4_PROBE_ST41[1,:],2)


In [ ]:
# Plot every station's m2 trend vs average. Look for weird straight lines

fig,axs = plt.subplots(24,3)

st = 0
for i in range(24):
    for j in range(3):
        plt.sca(axs[i,j])
        plt.plot(azi_avg_df.index.values,
                 azi_avg_df['m2'] - np.mean(azi_avg_df['m2']),
                 '.', color='navy')
        plt.plot(vtr_time_bin_df.index.values,
                 vtr_time_bin_df['st'+str(st)+',m2'] - np.mean(vtr_time_bin_df['st'+str(st)+',m2']),
                 '.', color='orange', markersize=1)
        plt.plot(fp_time_bin_df.index.values,
                 fp_time_bin_df['st'+str(st)+',m2'] - np.mean(fp_time_bin_df['st'+str(st)+',m2']),
                 '.', color='green', markersize=1)
        axs[i,j].set_title('st '+str(st)+': '+str(trfp.STATION_PROBE_NUMBER[st])+' probes')
        st += 1

fig.set_size_inches(12,96)
fig.tight_layout()

In [ ]:
# load all fixed probe runs moment_df into one big moment_df (runs 3959--3994)
print 'Appending fixed probe runs.'
fp_moment_df_2 = pd.read_hdf('60hr_fixed_probe_runs_new.h5', key='run_3959_moment_df')
for run in np.arange(3960, 3995):
    temp_df = pd.read_hdf('60hr_fixed_probe_runs_new.h5', key='run_'+str(run)+'_moment_df')
    print '\rAppending run ' + str(run) + '.',
    fp_moment_df_2 = fp_moment_df_2.append(temp_df)

In [ ]:
# recalculate 4-probe m2 moments from interp dataframes

# load all fixed probe runs moment_df into one big moment_df (runs 3959--3994)
print 'Appending fixed probe runs.'
fp_interp_df = pd.read_hdf('60hr_fixed_probe_runs_new.h5', key='run_3959_interp_df')
for run in range(3960, 3995):
    temp_df = pd.read_hdf('60hr_fixed_probe_runs_new.h5', key='run_'+str(run)+'_interp_df')
    print '\rAppending run ' + str(run) + '.',
    fp_interp_df = fp_interp_df.append(temp_df)

In [ ]:
st = 53

probes = trfp.STATION_PROBE_ID[st]
fps = ['fp'+str(probe) for probe in probes]
stm = 'st'+str(st)+',m2'

bins = np.arange(1524384055, 1524641055, 1000)-500
bin_centers = np.arange(1524384055, 1524640055, 1000)

interp_time_bin_df = fp_interp_df.groupby(pd.cut(fp_interp_df.index,bins)).mean()
interp_time_bin_df.index = bin_centers

moment_time_bin_df = fp_moment_df_2.groupby(pd.cut(fp_moment_df_2.index,bins)).mean()
moment_time_bin_df.index = bin_centers

m2_test =  interp_time_bin_df[fps].dot(np.transpose(trfp.THETA_FP_4))[1]

fig,axs = plt.subplots(2,1)

plt.sca(axs[0])
plt.plot(m2_test, '.', color='navy')
plt.plot(moment_time_bin_df[stm], '.', color='orange')

plt.sca(axs[1])
plt.plot(m2_test - moment_time_bin_df[stm], '.', color='navy')

fig.set_size_inches(12,8)
fig.tight_layout()